## Computation of persistent homology with R or Z/pZ coeficients

Suppose we are given a filtration of abstract simplicial complexes $K_1 \le K_2 \le \cdots \le K_n$, where each $K_i$ is given as a set of maximal simplices of $K_i$ that did no appear as simplices of the previous level of the filtration. Our task will be to determine the persistent homology groups of this filtration with coefficients in a field $F$, where $F$ is $\mathbb{R}$ or $\mathbb{Z}/p\mathbb{Z}$ for a prime $p$.

### Basic data manipulation

Our filtration will be biven as a list `K = [K1, K2, ..., Kn]` where each `Ki` is given as a list of maximal simplices as described above, e.g. `K = [[(1,), (3,)], [(2,4)], [(1,4), (2,3)], [(3,4)], [(1,2)], [(1,2,4)]]`. We present each simplex as a tuple (e.g. $\tau = (1,2,3)$). We will not build a full simplicial complex for each level of the filtration, but will only add new simplices as they appear at each level. Building on what we did earlier, we will present this as a list of 'dimension dictionaries', where each dictionary corresponds to a level of the filtration with only 'new' simplices. Each `key => value` pair is of the form $d$ `=>` list of 'new' simplices of dimension $d$ in $K_i$.

In [ ]:
# To build the boundary matrix (or matrices) we will need the filtration of the full simplicial
# complex Kn. We will represent each stage of the filtration with a 'dimension dictionary'  
# that contains all simplices of the current level Ki, which are not contained 
# in the previous levels.

# Compute the boundary of a simplex sx and represent it as a list of maximal simplices, 
# i.e., just determine the codimenstion 1 faces of sx. 
# E.g. for sx = (1, 2, 3) we get [(1, 2), (1, 3), (2, 3)].
function simplex_boundary(sx)
    # determine the codimension 1 simplices in the boundary of the simplex sx
    # 'remove' one entry of a tuple at each specific index by splicing the corresponding subtuples
    cd1faces = ([(sx[1:k-1]..., sx[k+1:end]...) for k in eachindex(sx)])
    return cd1faces
end

# Create a vector of 'dimension dictionaries' representing each level of the filtration
function dim_dict_filtration(max_filtration)
    # determine the dimension+1 of each level of the filtration
    N = [maximum([length(sx) for sx in Ki]) for Ki in max_filtration]
    # set up the dictionaries (empty vectors of k-tuples in each dimension)
    dict_filtration = [Dict((k-1) => Vector{NTuple{k, Int}}() for k in 1:n) for n in N]
    
    # fill in the dictionaries for each level i of the filtration
    for i in eachindex(N)
        # first, add all simplices in max_filtration[i] to dict_filtration[i] 
        # in appropriate dimensions
        for sx in max_filtration[i]
            # check if sx is already contained in previous levels of the filtration
            # (For proper inputs, these checks should not be needed.)
            is_contained = false
            for j in 1:(i-1)
                is_contained |= !isnothing(findfirst(==(sx), dict_filtration[j]))
                if is_contained
                    break
                end
            end
            # add the simplex and sort its vertices (this simplifies recognition of simplices later)
            if !is_contained
                push!(dict_filtration[i][length(sx)-1], Tuple(sort(collect(sx))))
            end
        end
        
        # second, add all faces of the simplices in max_filtration[i] 
        # (apart from those that are already contained in previous levels)
        for k in (N[i]-1):-1:1
            for sx in dict_filtration[i][k]
                for bsx in simplex_boundary(sx)
                    # check if bsx is contained in previous levels
                    is_contained = false
                    for j in 1:(i-1)
                        if length(bsx)-1 in keys(dict_filtration[j])
                           is_contained |= !isnothing(findfirst(==(bsx), dict_filtration[j][length(bsx)-1]))
                        end
                        # stop the search if already found in one of the previous levels
                        if is_contained
                            break
                        end
                    end
                    # if it is not contained (and not already in the current level), 
                    # add it to the current level
                    if !is_contained && isnothing(findfirst(==(bsx), dict_filtration[i][k-1]))
                        push!(dict_filtration[i][k-1], bsx)
                    end
                end
            end
        end
    end
    return dict_filtration
end

In [ ]:
# Let's test this.
K = [[(1,), (3,)], [(2,4)], [(1,4), (2,3)], [(3,4)], [(1,2)], [(1,2,4)]]
Kdict = dim_dict_filtration(K)

### Building the boundary matrix D

We will build a 'big' boundary matrix $D$ for the computation of persitent homology. We order the simplices according to the level of the filtration, and by dimension in each level of the filtration. This order then represents row and column index in the matrix $D$.

Since $D$ is a mostly zero matrix, we represent it as a sparse array. We need `using SparseArrays`.

In [ ]:
# an auxiliary function to find the 'index' of a simplex in the 'dimension dictionary'
# of a filtration
function simplex_index(sx, dim_dict_filt)
    # initialize index at 0
    index = 0
    # find sx in dim_dict_filt
    for k in eachindex(dim_dict_filt)
        if length(sx)-1 in keys(dim_dict_filt[k])
            l = findfirst(==(sx), dim_dict_filt[k][length(sx)-1])
            if !isnothing(l)
                # return the index as soon as the simplex is found
                return index + l + sum(Vector{Int}([length(dim_dict_filt[k][dim]) for dim in 0:(length(sx)-2)]))
            end
        end
        index += sum([length(dim_dict_filt[k][dim]) for dim in keys(dim_dict_filt[k])])
    end
end

# boundary matrix for persistent homology in one large matrix
function persistent_boundary(dim_dict_filt, p = 0)
    # check if p is 0 or prime
    if p != 0 && !isprime(p)
        throw(DomainError(p, "p should be 0 or a prime."))
    end
    # determine the size of this huge matrix
    N = sum([sum([length(cplx[key]) for key in keys(cplx)]) for cplx in dim_dict_filt])
    # use a sparse array for this mostly 0 matrix
    pbd_matrix = spzeros(p == 0 ? Float64 : Mod{p}, N, N)
    # fill this matrix with +/-1 at appropriate positions
    col_ind = 1
    for k in eachindex(dim_dict_filt)
        # Keys (which represent the dimension of a simplex in our case) aren't sorted by default.
        # We need to sort them.
        for dim in sort(collect(keys(dim_dict_filt[k])))
            for j in eachindex(dim_dict_filt[k][dim])
                # only simplices of dimension >=1 have nonempty boundary
                if length(dim_dict_filt[k][dim][j]) >= 2
                    # find the boundary of the current simplex
                    bsx = simplex_boundary(dim_dict_filt[k][dim][j])
                    for bsx_ind in eachindex(bsx)
                        # and add the corresponding +/-1 entry into the matrix
                        pbd_matrix[simplex_index(bsx[bsx_ind], dim_dict_filt), col_ind] = (-1)^(bsx_ind+j)
                    end
                end
                col_ind += 1
            end
        end
    end
    return pbd_matrix
end

In [ ]:
# Let's continue testing.
using SparseArrays, Mods, Primes
D = persistent_boundary(Kdict, 3)

## Persistent homology computation

To compute persistent homology groups (or the corresponding Betti numbers) we need to column-reduce the boundary matrix $D$. To keep track of the generators, we perform the same operations on an additional identity matrix.

In [ ]:
# column reduction (for a persistence boundary matrix)
# This function will reduce D in place and return a matrix G. The columns of G represent coefficients of a linear combination of corresponding simplices which represents 
# a generator of homology.
function column_reduce(D)
    # The matrix G will keep track of the generators. We start with an identity matrix of the same size 
    # (and same entry type) as D.
    G = sparse(1:size(D, 2), 1:size(D, 2), [eltype(D)(1) for _ in 1:size(D, 2)])
    # initiate the column index and iterate until all columns are exhausted
    j = 1
    while j <= size(D, 2)
        # find lowest nonzero entry in the current column
        k = findlast(!=(0), D[:, j])
        if isnothing(k)
            # go to the next column if no such entries
            j += 1
            continue
        end
        for jj in 1:(j-1)
            # find lowest nonzero entry in previous columns
            kk = findlast(!=(0), D[:, jj])
            # if one is found with the same row index as the lowest entry in the current column ...
            if !isnothing(kk) && kk == k
                # perform column reduction ... 
                scale = D[k, j]/D[k, jj]
                D[:, j] -= scale*D[:, jj]
                # ... do the same operation on G ...
                G[:, j] -= scale*G[:, jj]
                # ... and repeat
                break
            end
            # once all previous columns are exhausted ...
            if jj == j-1
                # ... go to the next column
                j += 1
            end
        end
    end
    # 'un-store' the zero entries that appeared in the sparse matrix D
    dropzeros!(D)
    return G
end

In [ ]:
# Another test.
column_reduce(D)